<a href="https://colab.research.google.com/github/kim-kidong/hello-world/blob/master/%EC%A3%BC%EC%8B%9D%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주식가격 예측(LSTM)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout,Dense,Activation
import datetime

# Data입력

In [0]:
data = pd.read_csv("kkkk.csv")
data.head()

# Compute Mid Price

In [0]:
high_prices = data['high'].values
low_prices = data['low'].values
mid_prices = (high_prices + low_prices)/2

#Create Windows

In [0]:
seq_len = 50
sequence_length = seq_len + 1


result = []
for index in range(len(mid_prices) - sequence_length):
  result.append(mid_prices[index:index + sequence_length])

# Normalize Data

In [0]:
normalized_data = []
for window in result:
  normalized_window = [((float(p)/float(window[0]))-1) for p in window]
  normalized_data.append(normalized_window)
  
  
result = np.array(normalized_data)


row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
y_train = train[:, :-1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
y_test = result[row:, :-1]

x_train.shape, x_test.shape

# Build a Model

In [0]:
model = sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50,1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

# Traing

In [0]:
model.fit(x_train, y_train,
  validation_data=(x_test, y_test),
  batch_size=10, epochs=20)

# Predition

In [0]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white')
ax = fig.subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='prediction')
ax.legend()
plt.show()